In [85]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter

/Users/andrew/src/otp-scheduler


In [86]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200415_final/restaurants.csv').set_index('code')
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415_final/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415_final/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

rest_usage = restaurants.copy()
rest_usage.iloc[:, 2:] = 0

rest_cap = restaurants.copy()

# ASSUMPTIONS
MEAL_CAPACITY = 20000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

curr_capacity = MEAL_CAPACITY

total_hospitals = hospitals.shape[0]

### Idea #1

1. Go in order of hospital priority + who has least meals served so far (start with VIPs?)
2. find the order request that’s the most meals (or a request that’s in the biggest unserved cluster) that hasn’t yet been fulfilled. 
3. Assign a restaurant that can handle that restaurant and hopefully everything else in the cluster. 
4. Adjust meal counts for other hospitals that will be delivered to in that cluster. 
5. Update meals for all hospitals, capacity for that restaurant, and repeat. 

In [87]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

meal_log = None
meal_log_key_kludge = set()

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            # Add only if this cell is not blank, AND we haven't already added this hospital/meal.
            # This is possible if this order exists in both the regular and VIP meal schedule.
            # Since we add the VIP meals first, then we're guaranteed for it to be counted as a VIP meal if it's in both places.
            if not pd.isnull(val) and (name, schedule.columns[j + 4]) not in meal_log_key_kludge:
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
                meal_log_key_kludge.add((name, schedule.columns[j + 4]))
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [88]:
meal_log

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
0,Boston Children's Hospital - ED,1,1,False,dinner_mon,60.0,NaN,mon,False
1,Boston Children's Hospital - ED,1,1,False,dinner_tues,60.0,NaN,tues,False
2,Boston Children's Hospital - ED,1,1,False,dinner_wed,60.0,NaN,wed,False
3,Boston Children's Hospital - ED,1,1,False,dinner_thurs,60.0,NaN,thurs,False
4,Boston Children's Hospital - ED,1,1,False,dinner_fri,60.0,NaN,fri,False
...,...,...,...,...,...,...,...,...,...
144,Massachusetts General Hospital - Central,3,1,True,lunch_sat,300.0,NaN,sat,False
145,Massachusetts General Hospital - Central,3,1,True,dinner_mon,500.0,NaN,mon,False
146,Massachusetts General Hospital - Central,3,1,True,dinner_wed,500.0,NaN,wed,False
147,Massachusetts General Hospital - Central,3,1,True,dinner_fri,500.0,NaN,fri,False


In [89]:
# 1. Fill outstanding VIP meals
# 2. Fill out with LCD approach (select min, if tie take one with better priority)

In [90]:
def maybe_compact(df, meal, compact):
    if compact:
        return df[df[meal].notnull()]
    return df

def get_rest_cap_cluster_for_meal(meal, compact=False):
    r = rest_cap[[meal]]
    return maybe_compact(r, meal, compact)

In [91]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, row['zone'])

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, row['zone'], target_quantity)
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        print('No suitable restaurants already working this cluster')
        # Couldn't find an appropriate restaurant already serving cluster
        
        restaurants_in_other_zones = meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] != row['zone']) & (meal_log['restaurant'].notnull())]['restaurant'].unique()
        
        print('filtering our candidates with two lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[(~r_cap.index.isin(restaurants_serving_cluster)) & (~r_cap.index.isin(restaurants_in_other_zones)) & (r_cap[meal] >= target_quantity)]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # Sort in quantity order (TODO KNOB: THIS USED TO BE ascending=False)
        r_candidates = r_candidates.sort_values(by=meal, ascending=True)
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                return i
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True

# HANDLE VIPS
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[row['hospital'], 'failed'] = True
    print()
    print() 

Assigning BIDMC - West Campus lunch_mon
BIDMC - West Campus lunch_mon 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 10
RT
debiting r capacity:  RT 150.0
increasing r usage RT 45.0
updating log BIDMC - West Campus lunch_mon RT


Assigning BIDMC - West Campus lunch_tues
BIDMC - West Campus lunch_tues 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 10
ME
debiting r capacity:  ME 100.0
increasing r usage ME 45.0
updating log BIDMC - West Campus lunch_tues ME


Assigning BIDMC - West Campus lunch_wed
BIDMC - West Campus lunch_wed 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 11
ME
debiting r

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 dinner_tues 1 60.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 9
FC
debiting r capacity:  FC 100.0
increasing r usage FC 60.0
updating log BIDMC - ICU West dinner_tues FC


Assigning BIDMC - ICU West dinner_thurs
BIDMC - ICU West dinner_thurs 1 60.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 9
FC
debiting r capacity:  FC 100.0
increasing r usage FC 60.0
updating log BIDMC - ICU West dinner_thurs FC


Assigning BIDMC - ICU East dinner_tues
BIDMC - ICU East dinner_tues 1 20.0
Checking for restaurants already in cluster Counter({'FC': 1})
Found an existing restaurant with capacity FC 40.0
Returning a restaurant with capacity, working on the same day FC 40.0
debiting r capacity:  FC 40.0
increasing r usage FC 20.0
updating log BID

Returning a restaurant with capacity, working on the same day MD 355.0
debiting r capacity:  MD 355.0
increasing r usage MD 12.0
updating log Boston Healthcare for the Homeless - Tents lunch_sun MD


Assigning Boston Healthcare for the Homeless - Tents dinner_mon
Boston Healthcare for the Homeless - Tents dinner_mon 2 12.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['SH']
Number of candidates with capacity 6
FC
debiting r capacity:  FC 100.0
increasing r usage FC 12.0
updating log Boston Healthcare for the Homeless - Tents dinner_mon FC


Assigning Boston Healthcare for the Homeless - Tents dinner_tues
Boston Healthcare for the Homeless - Tents dinner_tues 2 12.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['FC' 'SH']
Number of candidates with capacity 7
CS
debiting r capacity:  CS 100.0
increasing r u

Number of candidates with capacity 4
BB
debiting r capacity:  BB 400.0
increasing r usage BB 34.0
updating log BWH - Faulkner 7N Covid Unit dinner_sun BB


Assigning BWH - Faulkner ICU lunch_mon
BWH - Faulkner ICU lunch_mon 2 17.0
Checking for restaurants already in cluster Counter({'IG': 2, 'FC': 1})
Found an existing restaurant with capacity IG 170.0
Returning a restaurant with capacity, working on the same day IG 170.0
debiting r capacity:  IG 170.0
increasing r usage IG 17.0
updating log BWH - Faulkner ICU lunch_mon IG


Assigning BWH - Faulkner ICU lunch_tues
BWH - Faulkner ICU lunch_tues 2 17.0
Checking for restaurants already in cluster Counter({'IG': 2, 'FC': 1})
Found an existing restaurant with capacity IG 170.0
Returning a restaurant with capacity, working on the same day IG 170.0
debiting r capacity:  IG 170.0
increasing r usage IG 17.0
updating log BWH - Faulkner ICU lunch_tues IG


Assigning BWH - Faulkner ICU lunch_wed
BWH - Faulkner ICU lunch_wed 2 17.0
Checking for res

Number of candidates with capacity 6
SH
debiting r capacity:  SH 300.0
increasing r usage SH 70.0
updating log South Boston Community Health Center lunch_thurs SH


Assigning South Boston Community Health Center lunch_fri
South Boston Community Health Center lunch_fri 2 70.0
Checking for restaurants already in cluster Counter({'RT': 3, 'FC': 1, 'IG': 1})
Found an existing restaurant with capacity RT 103.0
Returning a restaurant with capacity, working on the same day RT 103.0
debiting r capacity:  RT 103.0
increasing r usage RT 70.0
updating log South Boston Community Health Center lunch_fri RT


Assigning Massachusetts General Hospital - ICU dinner_sat
Massachusetts General Hospital - ICU dinner_sat 3 140.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['SH' 'MM' 'MD']
Number of candidates with capacity 4
BB
debiting r capacity:  BB 400.0
increasing r usage BB 140.0
updating log Massachusetts Gen

debiting r capacity:  LD 500.0
increasing r usage LD 500.0
updating log Massachusetts General Hospital - Central dinner_fri LD


Assigning Massachusetts General Hospital - Central dinner_sun
Massachusetts General Hospital - Central dinner_sun 3 300.0
Checking for restaurants already in cluster Counter({'LD': 1})
Found an existing restaurant with capacity LD 360.0
Returning a restaurant with capacity, working on the same day LD 360.0
debiting r capacity:  LD 360.0
increasing r usage LD 300.0
updating log Massachusetts General Hospital - Central dinner_sun LD




In [92]:
# meal_log[meal_log.vip == True]

In [93]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index().iloc[0]
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    select_and_assign_hospital()
    print()
    print()

Next hospital: hospital         Boston Children's Hospital - ED
restaurant                                     0
priority_rank                                  1
Name: 0, dtype: object
Possible orders                           hospital  zone  priority_rank    vip          meal  \
0  Boston Children's Hospital - ED     1              1  False    dinner_mon   
1  Boston Children's Hospital - ED     1              1  False   dinner_tues   
2  Boston Children's Hospital - ED     1              1  False    dinner_wed   
3  Boston Children's Hospital - ED     1              1  False  dinner_thurs   
4  Boston Children's Hospital - ED     1              1  False    dinner_fri   

   quantity restaurant    day  failed  
0      60.0        NaN    mon   False  
1      60.0        NaN   tues   False  
2      60.0        NaN    wed   False  
3      60.0        NaN  thurs   False  
4      60.0        NaN    fri   False  
Assigning Boston Children's Hospital - ED dinner_mon
Boston Children's Hospita

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value


Assigning Newton Wellesley Hospital lunch_mon
Newton Wellesley Hospital lunch_mon 5 50.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['RT' 'MD' 'BB' 'PO' 'IG' 'FC' 'SH']
Number of candidates with capacity 3
SU
debiting r capacity:  SU 500.0
increasing r usage SU 50.0
updating log Newton Wellesley Hospital lunch_mon SU


Next hospital: hospital         Steward St. Elizabeth's Medical Center
restaurant                                            0
priority_rank                                         1
Name: 0, dtype: object
Possible orders                                    hospital  zone  priority_rank    vip  \
133  Steward St. Elizabeth's Medical Center     5              1  False   
134  Steward St. Elizabeth's Medical Center     5              1  False   
135  Steward St. Elizabeth's Medical Center     5              1  False   
136  Steward St. Elizabeth's Medical Center     5             

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

Checking for restaurants already in cluster Counter({'IG': 4, 'FC': 1, 'SH': 1})
Found an existing restaurant with capacity SH 230.0
Returning a restaurant with capacity, working on the same day SH 230.0
debiting r capacity:  SH 230.0
increasing r usage SH 120.0
updating log Codman Square Health Center - Site 1 lunch_mon SH


Next hospital: hospital         Codman Square Health Center - Site 2
restaurant                                          0
priority_rank                                       2
Name: 0, dtype: object
Possible orders                                 hospital  zone  priority_rank    vip  \
77  Codman Square Health Center - Site 2     2              2  False   
78  Codman Square Health Center - Site 2     2              2  False   
79  Codman Square Health Center - Site 2     2              2  False   
80  Codman Square Health Center - Site 2     2              2  False   
81  Codman Square Health Center - Site 2     2              2  False   

           meal  quanti

...

 IG 49.0
increasing r usage IG 20.0
updating log Dana Farber Cancer Institute - Inpatient PAs lunch_sun IG


Next hospital: hospital         Newton Wellesley Hospital
restaurant                               6
priority_rank                            1
Name: 0, dtype: object
Possible orders                       hospital  zone  priority_rank    vip          meal  \
126  Newton Wellesley Hospital     5              1  False    dinner_mon   
127  Newton Wellesley Hospital     5              1  False   dinner_tues   
128  Newton Wellesley Hospital     5              1  False    dinner_wed   
129  Newton Wellesley Hospital     5              1  False  dinner_thurs   
130  Newton Wellesley Hospital     5              1  False    dinner_fri   
131  Newton Wellesley Hospital     5              1  False    dinner_sat   
132  Newton Wellesley Hospital     5              1  False    dinner_sun   

     quantity restaurant    day  failed  
126      50.0        NaN    mon   False  
127      50.0  

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Next hospital: hospital         Massachusetts General Hospital - Central
restaurant                                              7
priority_rank                                           1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Newton Wellesley Hospital
restaurant                               7
priority_rank                            1
Name: 0, dtype: object
Possible orders                       hospital  zone  priority_rank    vip          meal  \
127  Newton Wellesley Hospital     5              1  False   dinner_tues   
128  Newton Wellesley Hospital     5              1  False    dinner_wed   
129  Newton Wellesley Hospital     5              1  False  dinner_thurs   
130  Newton Wellesley Hospital     5              1  False    dinner_fri   
131  Newton Wellesley Hospital     5              1  False    dinner_sat   
132  Newton Welles

18    breakfast_sun     200.0        NaN    sun   False  
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_mon
We cannot to afford this meal because of our global limit! 31.0 210.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_tues
We cannot to afford this meal because of our global limit! 31.0 210.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_wed
We cannot to afford this meal because of our global limit! 31.0 210.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_thurs
We cannot to afford this meal because of our global limit! 31.0 210.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_fri
We cannot to afford this meal because of our global limit! 31.0 210.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_sat
We cannot to afford this meal because of our global limit! 31.0 200.0
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_sun
We cannot to afford this meal because of our global limit! 31.0 200.0



/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [94]:
curr_capacity, hospitals_that_failed_placement

(7.0,
 {'BIDMC - Chelsea',
  'BIDMC - East Campus',
  'BIDMC - ICU East',
  'BIDMC - ICU West',
  'BIDMC - West Campus',
  'BWH - Faulkner 7N Covid Unit',
  'BWH - Faulkner ICU',
  "Boston Children's Hospital - ED",
  'Boston Healthcare for the Homeless - Albany Street',
  'Boston Healthcare for the Homeless - Tents',
  'Boston Medical Center',
  "Brigham & Women's Hospital (COVID ICUs)",
  "Brigham and Women's - Faulkner ED",
  "Brigham and Women's Hospital - ED",
  'Cambridge Health Alliance - Cambridge Hospital',
  'Cambridge Health Alliance - Everett Hospital',
  'Cambridge Health Alliance - Somerville Hospital',
  'Carney Hospital - ED',
  'Charles River Community Health',
  'Codman Square Health Center - Site 1',
  'Codman Square Health Center - Site 2',
  'Dana Farber Cancer Institute - Inpatient PAs',
  'Dimock Center',
  'Dot House Health Dorchester',
  'East Boston Neighborhood Health Center',
  'Harvard Street Neighborhood Health Center',
  'Massachusetts General Hospital - 

In [95]:
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

19993.0

In [96]:
# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan


# construct the double-wide output dataframe
for i, c in enumerate(hospitals.columns[4:]):
    output.insert((i*2+1)+4, c + '_rest', np.nan)
        
for i, row in meal_log.iterrows():
    output.loc[row['hospital'], row['meal']] = row['quantity']
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal'] + '_rest'] = row['restaurant']
    else:
        output.loc[row['hospital'], row['meal'] + '_rest'] = 'XXX'


# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')


In [97]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_mon_rest,breakfast_tues,breakfast_tues_rest,breakfast_wed,breakfast_wed_rest,...,dinner_wed,dinner_wed_rest,dinner_thurs,dinner_thurs_rest,dinner_fri,dinner_fri_rest,dinner_sat,dinner_sat_rest,dinner_sun,dinner_sun_rest
name,,,,,,,,,,,,,,,,,,,,,
BIDMC - West Campus,2,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - East Campus,3,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham and Women's Hospital - ED,4,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,FC,NaN,NaN,NaN,NaN,NaN,NaN,100.0,MM
Boston Children's Hospital - ED,18,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,SH,60.0,SH,60.0,SH,NaN,NaN,NaN,NaN
BIDMC - ICU West,21,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,60.0,FC,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - ICU East,22,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.0,FC,NaN,NaN,NaN,NaN,NaN,NaN
Dana Farber Cancer Institute - Inpatient PAs,23,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham & Women's Hospital (COVID ICUs),36,1,Longwood / Back Bay,1,210.0,XXX,210.0,XXX,210.0,XXX,...,210.0,SH,210.0,SH,210.0,SH,200.0,SH,200.0,SH
Whittier Street Health Center,15,2,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False).sum()

19993.0

In [99]:
hospitals.iloc[:, 4:].sum().sum()

24877.0

In [100]:
equity = meal_log.groupby(['priority_rank', 'hospital'])[['meal', 'vip', 'restaurant']].agg({
    'restaurant': [('orders_fulfilled', 'count')],
    'meal': [('orders_requested', 'count')],
    'vip': [('vip_orders', 'sum')],
})
equity

restaurant  \
                                                                 orders_fulfilled   
priority_rank hospital                                                              
1             BIDMC - East Campus                                               7   
              BIDMC - ICU East                                                  2   
              BIDMC - ICU West                                                  2   
              BIDMC - West Campus                                               7   
              BWH - Faulkner 7N Covid Unit                                     14   
              BWH - Faulkner ICU                                               14   
              Boston Children's Hospital - ED                                   5   
              Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Tents                       17   
              Boston Medical Center                                             8   
              Brigham & Women's Hospital (COVID ICUs)                          14   
              Brigham and Women's - Faulkner ED                                 4   
              Brigham and Women's Hospital - ED                                 4   
              Cambridge Health Alliance - Cambridge Hospital                    7   
              Cambridge Health Alliance - Everett Hospital                      6   
              Cambridge Health Alliance - Somerville Hospital                   7   
              Carney Hospital - ED                                              7   
              Dana Farber Cancer Institute - Inpatient PAs                      7   
              Massachusetts General Hospital - Central                          7   
              Massachusetts General Hospital - ED                               5   
              Massachusetts General Hospital - ICU                              2   
              Massachusetts General Hospital - PICU                            10   
              Mt. Auburn Hospital                                               1   
              Newton Wellesley Hospital                                         8   
              Steward St. Elizabeth's Medical Center                            7   
              Tufts Medical Center                                             10   
              VA Medical Center - West Roxbury                                  6   
2             BIDMC - Chelsea                                                   4   
              Charles River Community Health                                    4   
              Codman Square Health Center - Site 1                              4   
              Codman Square Health Center - Site 2                              4   
              Dimock Center                                                     4   
              Dot House Health Dorchester                                       4   
              East Boston Neighborhood Health Center                            6   
              Harvard Street Neighborhood Health Center                         4   
              Mattapan Community Health Center                                  3   
              North End Waterfront Health Center                                3   
              South Boston Community Health Center                              5   
              Southern Jamaica Plain Health Center (part of B...                4   
              Upham's Corner Health Center                                      3   
              Whittier Street Health Center                                     4   

                                                                             meal  \
                                                                 orders_requested   
priority_rank hospital                                                              
1             BIDMC - East Campus                                          

In [101]:
equity.to_csv('../../equity.csv')

In [102]:
meal_log[meal_log['hospital'] == 'Boston Medical Center']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
37,Boston Medical Center,2,1,False,lunch_sat,150.0,SU,sat,False
38,Boston Medical Center,2,1,False,lunch_sun,150.0,NaN,sun,True
39,Boston Medical Center,2,1,False,dinner_mon,400.0,NaN,mon,True
40,Boston Medical Center,2,1,False,dinner_tues,400.0,NaN,tues,True
41,Boston Medical Center,2,1,False,dinner_wed,400.0,NaN,wed,True
42,Boston Medical Center,2,1,False,dinner_thurs,400.0,NaN,thurs,True
43,Boston Medical Center,2,1,False,dinner_fri,400.0,NaN,fri,True
61,Boston Medical Center,2,1,True,lunch_mon,400.0,FC,mon,False
62,Boston Medical Center,2,1,True,lunch_tues,400.0,FC,tues,False
63,Boston Medical Center,2,1,True,lunch_wed,400.0,FC,wed,False


In [103]:
meal_log[meal_log['hospital'] == 'Massachusetts General Hospital - Central']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
142,Massachusetts General Hospital - Central,3,1,True,lunch_tues,500.0,SU,tues,False
143,Massachusetts General Hospital - Central,3,1,True,lunch_thurs,500.0,SU,thurs,False
144,Massachusetts General Hospital - Central,3,1,True,lunch_sat,300.0,BB,sat,False
145,Massachusetts General Hospital - Central,3,1,True,dinner_mon,500.0,LD,mon,False
146,Massachusetts General Hospital - Central,3,1,True,dinner_wed,500.0,LD,wed,False
147,Massachusetts General Hospital - Central,3,1,True,dinner_fri,500.0,LD,fri,False
148,Massachusetts General Hospital - Central,3,1,True,dinner_sun,300.0,LD,sun,False


In [109]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False)

code
SH    3262.0
LD    3022.0
MD    2653.0
FC    2422.0
BB    2223.0
SU    2024.0
PO    1050.0
IG    1020.0
PG     930.0
ME     581.0
RT     322.0
MM     312.0
CS     172.0
dtype: float64